# Convert HyperNeRF Data to NeRF-W Kubric Style

In [7]:
import numpy as np
from pathlib import Path
import json
import imageio
import shutil
import copy
import os

target_dir = Path('data/hypernerf/')
hypernerf_dir = Path('/rds/user/tw554/hpc-work/workspace/hypernerf/data/')
scene_id = 'kubric_soft'
target_data_dir = target_dir / f'{scene_id}_gt'
target_rgb_dir = target_data_dir / 'images'
target_cam_dir = target_data_dir / 'cameras'

target_cam_dir.mkdir(exist_ok=True, parents=True)
target_rgb_dir.mkdir(exist_ok=True, parents=True)

root_dir = hypernerf_dir / scene_id
img_scale = 2
# Where to save RGB images.
rgb_dir = root_dir / 'rgb-raw'
colmap_dir = root_dir / 'colmap'
camera_dir = root_dir / 'camera-gt'
test_dir = root_dir / 'freeze-test'
test_rgb_dir = test_dir / 'rgb-raw'
test_cam_dir = test_dir / 'camera-gt'


with open(root_dir / 'dataset.json','r') as f:
    dataset_data = json.load(f)
with open(root_dir / 'metadata.json','r') as f:
    time_data = json.load(f)

if (root_dir / 'scene_gt.json').exists():
    with open(root_dir / 'scene_gt.json','r') as f:
        scene_data = json.load(f)
else:
    with open(root_dir / 'scene.json','r') as f:
        scene_data = json.load(f)

# rgb_list = [rgb_dir / f'{idx}.png' for idx in dataset_data['ids']]
rgb_list = sorted(list(rgb_dir.glob('*.png'))) + sorted(list(test_rgb_dir.glob('*.png')))
valid_rgb_ids = list(range(len(rgb_list)))

cam_list = sorted(list(camera_dir.glob('*')))
cam_list += sorted(list(test_cam_dir.glob('*')))

# write dataset split
with open(target_data_dir / 'split.json','w') as f:
    split = {}
    split['train_ids'] = [rgb_list.index(rgb_dir / f'{idx}.png') for idx in dataset_data['train_ids']]
    split['val_ids'] = sorted(list(set(range(len(valid_rgb_ids))) - set(split['train_ids'])))
    print(f"Num train: {len(split['train_ids'])}\nNum val: {len(split['val_ids'])}")
    json.dump(split, f, indent=2)

Num train: 199
Num val: 100


In [8]:
# copy all the images and masks
    
for rgb in rgb_list:
    if 'freeze-test' in str(rgb):
        shutil.copy(str(rgb), str(target_rgb_dir / f'test_{rgb.stem}.png'))
    else:
        shutil.copy(str(rgb), str(target_rgb_dir))

for cam in cam_list:
    if 'freeze-test' in str(cam):
        shutil.copy(str(cam), str(target_cam_dir / f'test_{cam.stem}.json'))
    else:
        shutil.copy(str(cam), str(target_cam_dir))


img = imageio.imread(str(rgb))
image_size = [img.shape[1], img.shape[0]]
